In [1]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    NUM_WORKERS = 1
    PIN_MEMORY = True
    DATA_ORDER = "cw_ins"
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    
    # only flips
    MEAN_PIXEL = np.array([0.485, 0.456, 0.406],dtype=np.float32).reshape(1,1,-1)
    STD_PIXEL = np.array([0.229, 0.224, 0.225],dtype=np.float32).reshape(1,1,-1)
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    WIDTH = 224
    HEIGHT = 224
    CROP_SIZE = 224
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
train_image_dir = "/media/Data1/interns/aravind/train2017/"
# train_image_dir = "/media/Data1/interns/aravind/val2017/"
val_image_dir = "/media/Data1/interns/aravind/val2017/"
config = ProposalConfig()
model_dir = "./models/"
train_pickle = "/media/Data1/interns/aravind/re/data/train_cwid.pickle"
# train_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"
val_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"

In [4]:
# with open(train_pickle,"rb") as train_ann:
#     train_cwid = pickle.load(train_ann)
with open(val_pickle,"rb") as val_ann:
    val_cwid = pickle.load(val_ann)

In [5]:
# train_loader = model_lib.get_loader(train_cwid,config,train_image_dir)
val_loader = model_lib.get_loader(val_cwid,config,val_image_dir)

In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image
net = model_lib.MultiHGModel()

pretrained_dict = torch.load(model_dir+"iou_loss_01.pt")
net_dict = net.state_dict()


pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}
net_dict.update(pretrained_dict) 
net.load_state_dict(net_dict)

# net.vgg.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").vgg)
# net.classifier.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").classifier)
net = net.cuda()
with torch.no_grad():
    for i,data in enumerate(val_loader):
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = data
        print(config.CLASS_NAMES[np.argmax(batch_one_hot.numpy())])
        a,b,c,d= batch_images.numpy(), batch_impulses.numpy(),batch_gt_responses.numpy(),batch_bboxes.numpy()
        a = np.moveaxis(a,1,-1)
        b = np.moveaxis(b,1,-1)
        c = np.moveaxis(c,1,-1)
        d = np.moveaxis(d,1,-1)
        Image.fromarray(((a[0]*config.STD_PIXEL + config.MEAN_PIXEL)*255).astype(np.uint8),"RGB").show()
        Image.fromarray((b[0][:,:,0]*128).astype(np.uint8),"L").show()
        Image.fromarray((c[0][:,:,0]*128).astype(np.uint8),"L").show()
        Image.fromarray((d[0][:,:,0]*128).astype(np.uint8),"L").show()
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = batch_images.cuda(),batch_impulses.cuda(),batch_gt_responses.cuda(),batch_bboxes.cuda(),batch_one_hot.cuda()
        pred_class,pred_masks = net([batch_images,batch_impulses])
        pred_class = F.softmax(pred_class,dim=-1).squeeze()
#         class_acc = model_lib.class_acc(pred_class,batch_one_hot)
#         mask_acc = model_lib.mask_acc(pred_masks,batch_gt_responses)
        maxs, indices = torch.topk(pred_class,5,-1)
        pred_mask = pred_masks[1].squeeze()
        pred_mask = F.sigmoid(pred_mask)
#         pred_mask = F.threshold(pred_mask,0.7,0)
#         pred_mask = (pred_mask>0).float()
        pred_mask = pred_mask.squeeze().cpu().numpy()*255
        Image.fromarray(pred_mask.astype(np.uint8),"L").convert("RGB").show()
        pred_mask = pred_masks[0].squeeze()
        pred_mask = F.sigmoid(pred_mask)
#         pred_mask = F.threshold(pred_mask,0.5,0)
#         pred_mask = (pred_mask>0).float()
        pred_mask = pred_mask.squeeze().cpu().numpy()*128
        Image.fromarray(pred_mask.astype(np.uint8),"L").convert("RGB").show()

        for i in range(5):
            print(maxs[i],indices[i],config.CLASS_NAMES[int(indices[i])])
        input()

person
tensor(0.1832, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(0.1618, device='cuda:0') tensor(73, device='cuda:0') refrigerator
tensor(0.1435, device='cuda:0') tensor(70, device='cuda:0') oven
tensor(1.00000e-02 *
       6.6855, device='cuda:0') tensor(50, device='cuda:0') orange
tensor(1.00000e-02 *
       5.7165, device='cuda:0') tensor(54, device='cuda:0') pizza

microwave
tensor(0.5574, device='cuda:0') tensor(70, device='cuda:0') oven
tensor(0.2767, device='cuda:0') tensor(73, device='cuda:0') refrigerator
tensor(1.00000e-02 *
       3.8916, device='cuda:0') tensor(69, device='cuda:0') microwave
tensor(1.00000e-02 *
       3.8749, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-02 *
       2.7827, device='cuda:0') tensor(29, device='cuda:0') suitcase

person
tensor(0.4507, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       8.6805, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-02 *
       7.0551, device='cuda